In [ ]:
import os
import pandas as pd
from scipy.signal import savgol_filter
from sklearn.ensemble import IsolationForest

In [ ]:
#Set Directory; the folder where your csv and other data are, and you need to save data later
os.chdir('C:/Negar/Soil_DistrictsPaper')

In [ ]:
# Load CSV file for spectra data
csv_file_path = "C:\OSSL\OSSL-data\Sweden\LUCAS\data\soilsite.data.csv"  #location to the site data
df_site_data = pd.read_csv(csv_file_path)

In [ ]:
# === Step 1: Filtering using Savitzky-Golay ===
def apply_savitzky_golay(df, window_length=5, polyorder=2, mode='nearest'):
    """
    Apply Savitzky-Golay filter to smooth spectral data.
    
    :param df: DataFrame containing spectral data (wavelengths as columns)
    :param window_length: Window size for the filter (must be odd)
    :param polyorder: Polynomial order for smoothing
    :return: Smoothed DataFrame
    """
     # Ensure window_length is valid
    if df.shape[0] < window_length:
        raise ValueError(f"window_length ({window_length}) cannot be greater than the number of rows ({df.shape[0]})")

    # Apply Savitzky-Golay filter row-wise
    filtered_array = np.apply_along_axis(
        lambda row: savgol_filter(row, window_length, polyorder, mode="nearest"),
        axis=1,
        arr=df
    )

    # Convert back to DataFrame
    return pd.DataFrame(filtered_array, columns=df.columns, index=df.index)

# === Step 2: Scatter Correction using Standard Normal Variate (SNV) ===
def snv(input_data):
    """
    Applies Standard Normal Variate (SNV) to each row in the dataset.

    Parameters:
    - input_data (pd.DataFrame or np.ndarray): Spectral data.

    Returns:
    - pd.DataFrame: SNV-transformed data with the same shape as input.
    """

    # Ensure input is a NumPy array
    if isinstance(input_data, pd.DataFrame):
        input_array = input_data.to_numpy()
    else:
        input_array = input_data

    # Check for non-numeric data
    if not np.issubdtype(input_array.dtype, np.number):
        raise ValueError("SNV function received non-numeric data.")

    # Compute SNV (normalize each row)
    mean = np.mean(input_array, axis=1, keepdims=True)
    std_dev = np.std(input_array, axis=1, keepdims=True)
    snv_array = (input_array - mean) / std_dev

    # Convert back to DataFrame if original was a DataFrame
    if isinstance(input_data, pd.DataFrame):
        return pd.DataFrame(snv_array, columns=input_data.columns, index=input_data.index)
    return snv_array


# === Step 3: Compute First Derivatives ===
def compute_first_derivative(df):
    """
    Compute the first derivative of spectral data.
    
    :param df: DataFrame containing spectral data
    :return: First derivative DataFrame
    """
    if isinstance(df, np.ndarray):  # Convert to DataFrame if it became an array
        df = pd.DataFrame(df)

    # Get the derivatives
    derivative = df.diff(axis=1, periods=1)

    # Drop first column since diff creates NaN
    derivative.drop(axis=1, columns=derivative.columns[0], inplace=True)

    return derivative

# === Step 4: Remove Outliers using Isolation Forest ===
def remove_outliers(df, contamination=0.1):
    """
    Remove outliers using Isolation Forest.
    
    :param df: DataFrame containing spectral data
    :param contamination: Proportion of data expected to be outliers
    :return: Cleaned DataFrame
    """
    model = IsolationForest(contamination=contamination, random_state=42)
    labels = model.fit_predict(df)
    return df[labels == 1]  # Keep only inliers

In [ ]:
#you need to replace your dataframe with specified colomns here

spectral_data_savitzky = apply_savitzky_golay(spectral_data)
spectral_data_snv = snv(spectral_data_savitzky)
spectral_data_first_derivative = compute_first_derivative(spectral_data_snv)
spectral_data_cleaned = remove_outliers(spectral_data_first_derivative)